# Functions

_Also read [PEP 8 -- Style Guide for Python Code](https://www.python.org/dev/peps/pep-0008/)_

In [1]:
for i in range(3):
    print(i)
else:
    print("done!")  # won't be executed when iteration breaks calling the break statement

0
1
2
done!


## Defining functions

`return` is not required — any function returns `None` by default:

In [2]:
def foo():
    42

In [3]:
print(foo())

None


In [4]:
foo.__name__

'foo'

In [5]:
import dis

dis.dis('''
def foo():
    """I'm doing nothing and return 92."""
    return 92
foo()
''')

# Pay attention to the first 4 bytecode instructions

  2           0 LOAD_CONST               0 (<code object foo at 0x104e605b0, file "<dis>", line 2>)
              2 LOAD_CONST               1 ('foo')
              4 MAKE_FUNCTION            0
              6 STORE_NAME               0 (foo)

  5           8 LOAD_NAME                0 (foo)
             10 CALL_FUNCTION            0
             12 POP_TOP
             14 LOAD_CONST               2 (None)
             16 RETURN_VALUE

Disassembly of <code object foo at 0x104e605b0, file "<dis>", line 2>:
  4           0 LOAD_CONST               1 (92)
              2 RETURN_VALUE


## Documenting functions

- _[PEP 257 -- Docstring Conventions](https://www.python.org/dev/peps/pep-0257/)_
- _[Documentation — The Hitchhiker's Guide to Python](https://docs.python-guide.org/writing/documentation/)_
- _[Documenting Python Code: A Complete Guide](https://realpython.com/documenting-python-code/)_

We can use string literals to document functions. There are few different docstring formats: Google, NumPy/SciPy, reStructured Text, and Epytext.

In [6]:
def foo():
    """Returns some magic number"""
    return 42

In [7]:
foo.__doc__

'Returns some magic number'

In [8]:
help(foo)  # or foo? in IPython

Help on function foo in module __main__:

foo()
    Returns some magic number



## Keyword (named) arguments

In [9]:
def min(x, y):
    return x if x < y else y

In [10]:
min(-5, 12)

-5

In [11]:
# we can use few or all keyword arguments when calling a function

min(x=-5, y=12)
min(y=12, x=-5)  # order of keyword arguments is not important
min(-5, y=12)    # keyword arguments always follow positional arguments

-5

⚠️ Be careful using keyword arguments. If a function author renames them, it can break your code.

In [12]:
def min(a, b):
    ...
    
min(x=-5, y=12)

TypeError: min() got an unexpected keyword argument 'x'

## Mutable default arguments

In [13]:
# How default function arguments are built and stored

dis.dis('''
def foo(x=[], y=92):
    ...
''')

# Pay attention to the first 3 bytecode instructions

  2           0 BUILD_LIST               0
              2 LOAD_CONST               0 (92)
              4 BUILD_TUPLE              2
              6 LOAD_CONST               1 (<code object foo at 0x104f38710, file "<dis>", line 2>)
              8 LOAD_CONST               2 ('foo')
             10 MAKE_FUNCTION            1 (defaults)
             12 STORE_NAME               0 (foo)
             14 LOAD_CONST               3 (None)
             16 RETURN_VALUE

Disassembly of <code object foo at 0x104f38710, file "<dis>", line 2>:
  3           0 LOAD_CONST               0 (None)
              2 RETURN_VALUE


In [14]:
def unique(iterable, seen=set()):
    acc = []
    for item in iterable:
        if item not in seen:
            seen.add(item)
            acc.append(item)
    return acc

In [15]:
xs = [1, 1, 2, 3]
unique(xs)

[1, 2, 3]

In [16]:
unique(xs)

[]

In [17]:
unique.__defaults__

({1, 2, 3},)

How we can avoid this gotcha?

In [18]:
def unique(iterable, seen=None):
    seen = set(seen or [])
    acc = []
    for item in iterable:
        if item not in seen:
            seen.add(item)
            acc.append(item)
    return acc

In [19]:
xs = [1, 1, 2, 3]
unique(xs)

[1, 2, 3]

In [20]:
unique(xs)

[1, 2, 3]

In [21]:
unique.__defaults__

(None,)

## Required keyword arguments

In [22]:
def flatten(xs, depth=None):
    pass

In [23]:
flatten([1, [2], 3], depth=1)
flatten([1, [2], 3], 1)

In [24]:
# if you want to require keyword arguments in function calls
def flatten(xs, *, depth=None):
    pass

In [25]:
flatten([1, [2], 3], 1)

TypeError: flatten() takes 1 positional argument but 2 were given

In [26]:
# More explicit function call
flatten([1, [2], 3], depth=1)

## Packing and unpacking arguments

In [27]:
def min(*args):  # type(args) → class<'tuple'> — ordered and immutable data structure
    result = float('inf')
    for arg in args:
        if arg < result:
            result = arg
    return result

ℹ️ _Python idiom: `float('inf')`_. It acts as an unbounded upper value for comparison.

In [28]:
min(-5, 12, 13)

-5

In [29]:
min()

inf

Function with at least one required argument:

In [30]:
def min(first, *args):
    result = first
    for arg in args:
        if arg < result:
            result = arg
    return result

In [31]:
min()

TypeError: min() missing 1 required positional argument: 'first'

💡 _Unpacking works with any object implementing iterator protocol._

In [32]:
# set
# only for educational purposes, avoid unpacking sets – they're unordered!
xs = {-5, 12, 13}
min(*xs)

-5

In [33]:
# list
min(*[-5, 12, 13])

-5

In [34]:
# tuple
min(*(-5, 12, 13))

-5

⚠️ _Using unpacking, you always have to remember the order of function arguments._

---

In [35]:
def bounded_min(first, *args, lo=float('-inf'), hi=float('inf')):
    result = hi
    for arg in (first,) + args:
        if arg < result and lo < arg < hi:
            result = arg
    return max(result, lo)

In [36]:
bounded_min(-5, 12, 13)

-5

In [37]:
bounded_min(-5, 12, 13, lo=0, hi=255)

12

In [38]:
bounded_min(-5, 12, 13, lo=100, hi=200)

200

ℹ️ _Python idiom: `lo < arg < hi`._ It simplifies the condition.

## Keyword arguments (kwargs)

In [39]:
def runner(cmd, **kwargs):  # type(kwargs) → class<'dict'> — mutable data structure
    if kwargs.get('verbose', True):
        print('Logging enabled')
    else:
        print('Logging disabled')

In [40]:
runner('dramatiq', processes=4, threads=4)

Logging enabled


In [41]:
runner('dramatiq', processes=4, threads=4, verbose=False)

Logging disabled


In [42]:
runner('dramatiq', **{'processes': 2, 'threads': 2, 'verbose': False})

Logging disabled


In [43]:
options = {'processes': 16, 'threads': 16, 'verbose': False}
runner('dramatiq', **options)

Logging disabled


## Unpacking and assignment

In [44]:
acc = []
seen = set()

In [45]:
# using unpacking with assignment
(acc, seen) = ([], set())

In [46]:
x, y, z = [1, 2, 3]
x, y, z = {1, 2, 3} # unordered!
x, y, z = "123"

In [47]:
x

'1'

In [48]:
y

'2'

In [49]:
z

'3'

Parentheses are not required, but sometimes they can be useful:

In [50]:
rectangle = (0, 0), (4, 4)
(x1, y1), (x2, y2) = rectangle

[PEP 3132 -- Extended Iterable Unpacking](https://www.python.org/dev/peps/pep-3132/)

In [51]:
first, *rest = range(1, 5)

In [52]:
first, rest

(1, [2, 3, 4])

In [53]:
first, *rest, last = range(1, 5)
last

4

In [54]:
rest

[2, 3]

In [55]:
first, *rest, last = [42]

ValueError: not enough values to unpack (expected at least 2, got 1)

In [56]:
line = "Tesla S,€60 000,black,US"
name, price, *_ = line.split(",")  # OK!
name, price

('Tesla S', '€60 000')

---

In [57]:
import dis
dis.dis("first, *rest, last = ('a', 'b', 'c')")

  1           0 LOAD_CONST               0 (('a', 'b', 'c'))
              2 EXTENDED_ARG             1
              4 UNPACK_EX              257
              6 STORE_NAME               0 (first)
              8 STORE_NAME               1 (rest)
             10 STORE_NAME               2 (last)
             12 LOAD_CONST               1 (None)
             14 RETURN_VALUE


As we see, variable assignment works from left to right.

In [58]:
x, (x, y) = 1, (2, 3)

In [59]:
x

2

`x` is equal 2 as expected.

P.S. We used variable `x` twice in the expression just for educational purposes. This could be confusing for others.

In [60]:
dis.dis("first, *rest, last = ['a', 'b', 'c']")

  1           0 BUILD_LIST               0
              2 LOAD_CONST               0 (('a', 'b', 'c'))
              4 LIST_EXTEND              1
              6 EXTENDED_ARG             1
              8 UNPACK_EX              257
             10 STORE_NAME               0 (first)
             12 STORE_NAME               1 (rest)
             14 STORE_NAME               2 (last)
             16 LOAD_CONST               1 (None)
             18 RETURN_VALUE


💡 _Unpacking the list works differently on runtime._

---

In [61]:
def f(*args, **kwargs):
    pass

In [63]:
f(1, 2, 3, **{"foo": 42})

In [64]:
first, *rest = range(4)

In [65]:
for first, *rest in [range(4), range(2)]:
    pass

[PEP 448 -- Additional Unpacking Generalizations](https://www.python.org/dev/peps/pep-0448/) → Python 3.5

In [66]:
def f(*args, **kwargs):
    print(args, kwargs)

In [67]:
f(1, 2, *[3, 4], *[5], foo="bar", **{"baz": 42}, boo=24)

(1, 2, 3, 4, 5) {'foo': 'bar', 'baz': 42, 'boo': 24}


In [68]:
defaults = {"host": "0.0.0.0", "port": 8080}

In [69]:
{**defaults, "port": 80}

{'host': '0.0.0.0', 'port': 80}

In [70]:
[*range(5), 6]

[0, 1, 2, 3, 4, 6]

## Function scope

Functions are first-class citizens in Python:

In [71]:
def wrapper():
    def identity(x):
        return x
    return identity

In [72]:
f = wrapper()
f(42)

42

In [73]:
def make_min(*, lo, hi):
    """Returns configured min function"""
    def inner(first, *args):
        result = hi
        for arg in (first, ) + args:
            if arg < result and lo < arg < hi:
                result = arg
        return max(result, lo)
    return inner

In [74]:
bounded_min = make_min(lo=0, hi=255)
bounded_min(-5, 12, 13)

12

[Python scoping: understanding LEGB](https://blog.mozilla.org/webdev/2011/01/31/python-scoping-understanding-legb/)

In [75]:
type(globals()), type(locals())

(dict, dict)

An interpreter can define local variables during bytecode compilation and store them more effectively in an array. Therefore access to their values using indexes will be quicker than lookup by keys in globals.

In [76]:
def foo():
    def bar():
        return x
    print(bar.__closure__)
    print(locals())
    x = 92
    print(bar.__closure__)
    print(locals())
    
foo()

(<cell at 0x104f19d90: empty>,)
{'bar': <function foo.<locals>.bar at 0x104e6b9d0>}
(<cell at 0x104f19d90: int object at 0x102d1f4d0>,)
{'bar': <function foo.<locals>.bar at 0x104e6b9d0>, 'x': 92}


- local, `locals()` — by index
- enclosing, `foo.__closure__` — by index
- global, `foo.__globals__`, `globals()` — lookup in dict
- built-ins

In [77]:
min       # builtin
min = 42  # global

def f(*args):
    min = 2
    def g():     # enclosing
        min = 4  # local
        print(min)

---

In [78]:
min = 42   # globals()['min'] = 42
globals()  # globals is mutable dict

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  'for i in range(3):\n    print(i)\nelse:\n    print("done!")  # won\'t be executed when iteration breaks calling the break statement',
  'def foo():\n    42',
  'print(foo())',
  'foo.__name__',
  'import dis\n\ndis.dis(\'\'\'\ndef foo():\n    """I\'m doing nothing and return 92."""\n    return 92\nfoo()\n\'\'\')\n\n# Pay attention to the first 4 bytecode instructions',
  'def foo():\n    """Returns some magic number"""\n    return 42',
  'foo.__doc__',
  'help(foo)  # or foo? in IPython',
  'def min(x, y):\n    return x if x < y else y',
  'min(-5, 12)',
  '# we can use few or all keyword arguments when calling a function\n\nmin(x=-5, y=12)\nmin(y=12, x=-5)  # order of keyword arguments is not important\nmin(-5

In [79]:
def f():
    min = 2  # locals['min'] = 2
    print(locals())
    

f()

{'min': 2}


ℹ️ Function can use variables defined outside of function scope.

⚠️ _Lookup of variables happens during runtime:_

In [80]:
def f():
    print(i)

In [81]:
for i in range(4):
    f()

0
1
2
3


⚠️ The LEGB rule doesn't work for assignment:

In [82]:
min = 42
def f():
    min += 1   # <- lookup in a global scope!
    return min

In [83]:
f()

UnboundLocalError: local variable 'min' referenced before assignment

This behavior can be changed by using `global` and `nonlocal` operators.

In [84]:
min = 42
def f():
    global min
    min += 1   # <- lookup in a global scope!
    return min

In [85]:
f()
f()

44

⚠️ _Try to avoid global variables._

[PEP 3104 -- Access to Names in Outer Scopes](https://www.python.org/dev/peps/pep-3104/)

In [86]:
def cell(value=None):
    def get():
        return value
    def set(update):
        nonlocal value
        value = update
    return get, set

In [87]:
get, set = cell()

In [88]:
set(42)

In [89]:
get()

42

## Elements of functional programming

In [90]:
range(3)

range(0, 3)

In [91]:
list(range(3))

[0, 1, 2]

### Anonymous functions

In [92]:
lambda name: print("Hi " + name + "!")

<function __main__.<lambda>(name)>

In [93]:
lambda name, *args, age=30, **kwargs: 42

<function __main__.<lambda>(name, *args, age=30, **kwargs)>

In [94]:
f = lambda x: x + 1 # antipattern, don't define named functions using lambdas

### map, filter and zip

In [95]:
def identity(x):
    return x

In [96]:
map(identity, range(4))

In [97]:
list(map(identity, range(4)))

[0, 1, 2, 3]

In [98]:
set(map(lambda x: x % 7, [1, 9, 16, -1, 2, 5]))

In [99]:
list(map(lambda x, n: x ** n, [2, 3], range(1, 8)))

[2, 9]

In [100]:
xs = [0, 1, 2]
ys = [3, 4, 5, 6]

list(map(lambda x, y: x + y, xs, ys))  # the last item of the second list is not used

[3, 5, 7]

In [101]:
assert len(xs) == len(ys)  # good practice to prevent the situation from code above

AssertionError: 

In [102]:
# probably the only case when the map function can be really helpful — in other cases just use list comprehensions
(x, y, z) = map(str, xs)

In [103]:
x, y, z

('0', '1', '2')

In [104]:
xs = ["0", "1", "2", "3"]
(x, y, z) = map(int, xs)

ValueError: too many values to unpack (expected 3)

---

In [105]:
filter(lambda x: x % 2 != 0, range(10))

In [106]:
list(filter(lambda x: x % 2 != 0, range(10)))

[1, 3, 5, 7, 9]

In [107]:
xs = [0, None, [], {}, "", 42]
# returns truthy values
list(filter(None, xs))

[42]

---

In [108]:
list(zip("abc", range(3), [42j, 42j, 42j]))

[('a', 0, 42j), ('b', 1, 42j), ('c', 2, 42j)]

In [109]:
list(zip("abc", range(10)))

[('a', 0), ('b', 1), ('c', 2)]

### Comprehensions

Comprehensions come from the ABC language. They are a good alternative to the functional style. Almost always prefer comprehensions over the functions with lambdas, e.g. `map`, `filter`, `zip`.

#### Lists

In [110]:
[x ** 2 for x in range(10) if x % 2 == 1]

[1, 9, 25, 49, 81]

In [111]:
# compare to the functional style
list(map(lambda x: x ** 2,
        filter(lambda x: x % 2 == 1,
              range(10))))

[1, 9, 25, 49, 81]

In [112]:
nested = [range(5), range(8, 10)]
[x for xs in nested for x in xs]  # flatten

[0, 1, 2, 3, 4, 8, 9]

⚠️ _Nested comprehensions are hard to understand._

In [113]:
# don't write complicated expressions like this one
[(x, y)
 for x in range(5)
 if x % 2 == 0
 for y in range(x)
 if y % 2 == 1
]

[(2, 1), (4, 1), (4, 3)]

#### Sets and dicts

In [114]:
{x % 7 for x in [1, 9, 16, -1, 2, 5]}  # new set

{1, 2, 5, 6}

In [115]:
date = {'year': 2020, 'month': 'March', 'day': ''}
{k: v for k, v in date.items() if v}  # new dict without empty values

{'year': 2020, 'month': 'March'}

In [116]:
{x: x**2 for x in range(4)}

{0: 0, 1: 1, 2: 4, 3: 9}

In [117]:
(x ** 2 for x in range(5))

<generator object <genexpr> at 0x104f91970>

In [118]:
# there is very little difference between generator object and map object — the map function can iterate over multiple iterables
map(lambda x: x ** 2, range(5))